## 代码介绍

# *特征处理*
- 处理关闭的商店和停售的商品
- 融合商店数据集和商品数据集特征
- 添加销量特征的历史特征
- 对类别特征进行独热编码、类别编码
# *模型训练* 
- 设置训练集验证集测试集
- 设置模型参数训练模型
- 特征重要性画图
- 预测未来销量结果

---
# 模型介绍
LightGBM（Light Gradient Boosting Machine）是一款基于决策树算法的分布式梯度提升框架。它是一个快速高效、低内存占用、高准确度、支持并行和大规模数据处理的数据科学工具。

基本原理是：   
* 1. 直方图算法*      
先把连续的浮点特征值离散化成  个整数，同时构造一个宽度为  的直方图。在遍历数据的时候，根据离散化后的值作为索引在直方图中累积统计量，当遍历一次数据后，直方图累积了需要的统计量，然后根据直方图的离散值，遍历寻找最优的分割点。   
* 2. 直方图作差加速*   
一个叶子的直方图可以由它的父亲节点的直方图与它兄弟的直方图做差得到，在速度上可以提升一倍。通常构造直方图时，需要遍历该叶子上的所有数据，但直方图做差仅需遍历直方图的k个桶。在实际构建树的过程中，LightGBM还可以先计算直方图小的叶子节点，然后利用直方图做差来获得直方图大的叶子节点，这样就可以用非常微小的代价得到它兄弟叶子的直方图。   
* 3. 带深度限制的按叶子生长策略*   
LightGBM采用Leaf-wise的增长策略，该策略每次从当前所有叶子中，找到分裂增益最大的一个叶子，然后分裂，如此循环。因此同Level-wise相比，Leaf-wise的优点是：在分裂次数相同的情况下，Leaf-wise可以降低更多的误差，得到更好的精度；Leaf-wise的缺点是：可能会长出比较深的决策树，产生过拟合。因此LightGBM会在Leaf-wise之上增加了一个最大深度的限制，在保证高效率的同时防止过拟合。   
* 4. 单边梯度采样算法*    
  根据样本的权重信息对样本进行抽样，减少了大量梯度小的样本，但是还能不会过多的改变数据集的分布
* 5. 特征值捆绑算法*    
  输入：特征F，最大冲突数K，图G；
  输出：特征捆绑集合bundles；
  （1）构造一个边带有权重的图，其权值对应于特征之间的总冲突；
  （2）通过特征在图中的度来降序排序特征；
  （3）检查有序列表中的每个特征，并将其分配给具有小冲突的现有bundling（由γγ控制），或创建新bundling。  

优点：
  1. 速度更快   
  LightGBM 采用了直方图算法将遍历样本转变为遍历直方图，极大的降低了时间复杂度；
  LightGBM 在训练过程中采用单边梯度算法过滤掉梯度小的样本，减少了大量的计算；
  LightGBM 采用了基于 Leaf-wise 算法的增长策略构建树，减少了很多不必要的计算量；
  LightGBM 采用优化后的特征并行、数据并行方法加速计算，当数据量非常大的时候还可以采用投票并行的策略；
  LightGBM 对缓存也进行了优化，增加了缓存命中率；

2. 内存更小   
  使用直方图算法将特征值转变为 bin 值，且不需要记录特征到样本的索引，极大的减少了内存消耗；
在训练过程中采用互斥特征捆绑算法减少了特征数量，降低了内存消耗



---
# 特征处理

1.处理关闭的商店和停售的商品

In [ ]:
#处理关闭的商店和停售的商品
shop_sales_monthly = filtered.pivot_table(index='date_block_num',
                                          columns='shop_id',
                                          values='item_cnt_day',
                                          fill_value=0,
                                          aggfunc=sum)
shop_open_month_cnt = (shop_sales_monthly.iloc[-6:] >  0).sum() 
 # 有销量的记录
shop_open_month_cnt.head()  
# 每个店铺最后半年里有几个月有销量
shop_c_n = shop_sales_monthly[shop_open_month_cnt[shop_open_month_cnt < 6].index]
shop_c_n.tail(12)
# 最后半年经营月数少于6个月的店铺
item_selling_month_cnt = (item_sales_monthly.iloc[-6:] >  0).sum() 
item_selling_month_cnt.head()  # 这些商品在最后半年有几个月有销量
open_shop = shop_sales_monthly[shop_open_month_cnt[shop_open_month_cnt == 6].index]
open_shop.tail(7) # 最后半年都正常经营的商店
item_zero = item_sales_monthly[item_selling_month_cnt[item_selling_month_cnt == 0].index]
# 这些商品在最后半年都没有销量
item_zero.tail(12)
selling_item = item_sales_monthly[item_selling_month_cnt[item_selling_month_cnt > 0].index]
selling_item.tail(12)  # 最后半年有销量的商品
#只保留最后6个月正常经营的商店和有销售的商品
cl_set = filtered[filtered['shop_id'].isin(open_shop.columns) & filtered['item_id'].isin(selling_item.columns)]
cl_set
#统计月销量
from itertools import product
import time
ts = time.time()
martix = []
for i in range(34):
    record = cl_set[cl_set['date_block_num'] == i]
    group = product([i],record.shop_id.unique(),record.item_id.unique())
    martix.append(np.array(list(group)))
            
cols = ['date_block_num', 'shop_id', 'item_id']
martix = pd.DataFrame(np.vstack(martix), columns=cols)

martix

group = filtered.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day': np.sum})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)
group

martix = pd.merge(martix, group, on=['date_block_num', 'shop_id', 'item_id'], how='left')
martix.head()

test['date_block_num'] = 34
test['item_cnt_month'] = 0
martix = pd.concat([martix.fillna(0), test.drop(columns='ID')], sort=False, ignore_index=True, keys=['date_block_num','shop_id','item_id'])
martix


2.融合商店数据集和商品数据集特征

In [ ]:

#融合商店数据集和商品数据集特征
martix = martix.merge(shops[['shop_id', 'shop_type_code', 'shop_city_code']], on='shop_id', how='left')
martix = martix.merge(items.drop(columns='item_name'), on='item_id', how='left')
martix

#添加具体的年份和月份
martix['year'] =  martix['date_block_num'].map(lambda x: x // 12 + 2013)
martix['month'] = martix['date_block_num'].map(lambda x: x % 12)
martix.head()

# 商品 月销量均值
group = martix.groupby(['date_block_num','item_id']).agg({'item_cnt_month':'mean'})
group.columns = ['item_cnt_month_avg']
group.reset_index(inplace=True)
martix = martix.merge(group, on=['date_block_num', 'item_id'], how='left')
martix.head()

# 商店 月销量均值
group = martix.groupby(['date_block_num','shop_id']).agg({'item_cnt_month':'mean'})
group.columns = ['shop_cnt_month_avg']
group.reset_index(inplace=True)
martix = martix.merge(group, on=['date_block_num', 'shop_id'], how='left')
martix.head()

# 类别 月销量均值
group = martix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month':'mean'})
group.columns = ['cat_cnt_month_avg']
group.reset_index(inplace=True)
martix = martix.merge(group, on=['date_block_num', 'item_category_id'], how='left')
martix.head()

# 商店-类别 月销量均值
group = martix.groupby(['date_block_num','shop_id','item_category_id']).agg({'item_cnt_month':'mean'})
group.columns = ['shop_cat_cnt_month_avg']
group.reset_index(inplace=True)
martix = martix.merge(group, on=['date_block_num','shop_id','item_category_id'], how='left')
martix.head()

# 大类 月销量均值
group = martix.groupby(['date_block_num', 'item_type_code']).agg({'item_cnt_month':'mean'})
group.columns = ['itemtype_cnt_month_avg']
group.reset_index(inplace=True)
martix = martix.merge(group, on=['date_block_num', 'item_type_code'], how='left')
martix.head()

# 小类 月销量均值
group = martix.groupby(['date_block_num', 'sub_type_code']).agg({'item_cnt_month':'mean'})
group.columns = ['subtype_cnt_month_avg']
group.reset_index(inplace=True)
martix = martix.merge(group, on=['date_block_num','sub_type_code'], how='left')
martix.head()

# 城市-商品 月销量均值
group = martix.groupby(['date_block_num','shop_city_code','item_id']).agg({'item_cnt_month':'mean'})
group.columns = ['city_item_cnt_month_avg']
group.reset_index(inplace=True)
martix = martix.merge(group, on=['date_block_num','shop_city_code','item_id'], how='left')
martix.head()

# 商店类型-商品 月销量均值
group = martix.groupby(['date_block_num','shop_type_code','item_id']).agg({'item_cnt_month':'mean'})
group.columns = ['shoptype_item_cnt_month_avg']
group.reset_index(inplace=True)
martix = martix.merge(group, on=['date_block_num','shop_type_code','item_id'], how='left')
martix.head()

del group
gc.collect()


3.添加销量特征的历史特征

In [ ]:

#添加销量特征的历史特征
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

martix = lag_feature(martix, [1,2,3,6,12], 'item_cnt_month')
martix.head()

martix = lag_feature(martix, [1,2,3,6,12], 'item_cnt_month_avg')
martix = lag_feature(martix, [1,2,3,6,12], 'shop_cnt_month_avg')
martix.head()

martix.drop(columns=[ 'item_cnt_month_avg', 'shop_cnt_month_avg'], inplace=True)  # 只保留特征的历史信息
gc.collect()

martix = lag_feature(martix, [1,2,3,6,12], 'cat_cnt_month_avg')
martix = lag_feature(martix, [1,2,3,6,12], 'shop_cat_cnt_month_avg')
martix.head()

martix.drop(columns=['cat_cnt_month_avg', 'shop_cat_cnt_month_avg'], inplace=True)

martix = lag_feature(martix, [1,2,3,6,12], 'itemtype_cnt_month_avg')
martix = lag_feature(martix, [1,2,3,6,12], 'subtype_cnt_month_avg')
martix.head()

martix.drop(columns=['itemtype_cnt_month_avg', 'subtype_cnt_month_avg'], inplace=True)

martix = lag_feature(martix, [1,2,3,6,12], 'city_item_cnt_month_avg')
martix = lag_feature(martix, [1,2,3,6,12], 'shoptype_item_cnt_month_avg')
martix.head()

martix.drop(columns=[ 'city_item_cnt_month_avg','shoptype_item_cnt_month_avg'], inplace=True)
martix

martix[martix.columns[:20]].isna().any()

#使用之前月份的销量，会导致有很多记录缺失信息。需要将这部分缺失信息的记录去掉。

#前面延迟了12个月的销量信息，这里就直接把前12个月的记录删除。
train_set = martix[martix['date_block_num'] > 11].fillna(0)
train_set


4.对类别特征进行独热编码、类别编码

In [ ]:

#对类别特征进行独热编码、类别编码，将默认数据类型改为int32和float32，降低模型训练的内存压力
for col in train_set.columns:
    if col.find('code') >= 0:
        train_set[col] = train_set[col].astype(np.int8)
    elif train_set[col].dtype == 'float64':
        train_set[col] = train_set[col].astype(np.float32)
    elif train_set[col].dtype == 'int64':
        train_set[col] = train_set[col].astype(np.int16)
        
train_set['item_type_code'] = train_set['item_type_code'].astype('category')
train_set['sub_type_code'] = train_set['sub_type_code'].astype('category')
train_set.info()


# 模型训练

1.设置训练集验证集测试集

In [ ]:

import lightgbm as lgb

#前32个月作为训练集
#第33个月作为验证集，34月作为测试集
X_train = train_set[train_set['date_block_num'] < 33].drop(columns=['item_cnt_month'])  
# 训练集的样本特征
Y_train = train_set[train_set['date_block_num'] < 33]['item_cnt_month']  
# 训练集的样本标签

X_validate = train_set[train_set['date_block_num'] == 33].drop(columns=['item_cnt_month']) 
 # 校对集
Y_validate = train_set[train_set['date_block_num'] == 33]['item_cnt_month']

X_test = train_set[train_set['date_block_num'] == 34].drop(columns=['item_cnt_month'])  
# 测试集

del train_set
gc.collect()

# 把数据加载为模型适合的数据格式
train_data = lgb.Dataset(data=X_train, label=Y_train)
validate_data = lgb.Dataset(data=X_validate, label=Y_validate)



2.设置模型参数训练模型

In [ ]:

# 设置模型训练参数
import time
ts = time.time()
params = {"objective" : "regression",#目标函数：回归
         "metric" : "rmse", #评估函数均方根误差
         'n_estimators':10000, #随机森林中的分类器个数
         'early_stopping_rounds':50,#早停循环参数
        "num_leaves" : 200, #叶子节点数
        "learning_rate" : 0.01,#学习速率 
        "bagging_fraction" : 0.9,#建树的样本采样比例
        "feature_fraction" : 0.3, #建树的特征选择比例
        "bagging_seed" : 0}#bagging的随机种子数
print('Start....', ts)
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, validate_data], verbose_eval=1000) 
print('End...', time.time() - ts)



lgb_model.save_model('model_bestscore02.txt')  # 保存模型



3.特征重要性画图

In [ ]:
# 特征重要性画图
lgb.plot_importance(lgb_model, max_num_features=40, figsize=(12, 8))
plt.title("Featurertances")
plt.show()


4.预测未来销量结果

In [ ]:
# 根据项目要求，把数据“裁剪”到[0,20]的区间。
Y_test = lgb_model.predict(X_test).clip(0, 20)
Y_test

X_test['item_cnt_month'] = Y_test
X_test

#将预测结果合并到测试集
result = pd.merge(test[['ID', 'shop_id', 'item_id']],X_test[['shop_id','item_id','item_cnt_month']], on=['shop_id', 'item_id'], how='left')
result

result.isna().any()

result[result.shop_id.isin(shop_c_n.columns)]['shop_id'].unique()

#前面分析关闭的商店中，有3个商在最近半年里只有最后一个月有销量，推断是新开的商店。
#还有一些商品是最近半年都没有销量，推断是已经下架的商品，预测值填为0 
result.loc[result.item_id.isin(item_zero), 'item_cnt_month'] = 0
result.loc[result.item_id.isin(item_zero), 'item_cnt_month']

result[['ID','item_cnt_month']].to_csv('fromfinal01.csv',sep=',',index=False)